In [3]:
import pandas as pd
import numpy as np
from ipywidgets import interact
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

#Necessary for diaplaying all outputs; otherwise only gsd.colums will be displayed. 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## How advertising campaigns impact the different categories people?

In [4]:
#Loading the dataframes
products=pd.read_csv("data/products.csv")
transactions=pd.read_csv("data/transactions.csv")
stores=pd.read_csv("data/stores.csv")

In [9]:
# howing the heads of the data frames
products.head()
transactions.head()
stores.head()

,UPC,DESCRIPTION,MANUFACTURER,CATEGORY,SUB_CATEGORY,PRODUCT_SIZE
0,1111009477,PL MINI TWIST PRETZELS,PRIVATE LABEL,BAG SNACKS,PRETZELS,15 OZ
1,1111009497,PL PRETZEL STICKS,PRIVATE LABEL,BAG SNACKS,PRETZELS,15 OZ
2,1111009507,PL TWIST PRETZELS,PRIVATE LABEL,BAG SNACKS,PRETZELS,15 OZ
3,1111035398,PL BL MINT ANTSPTC RINSE,PRIVATE LABEL,ORAL HYGIENE PRODUCTS,MOUTHWASHES (ANTISEPTIC),1.5 LT
4,1111038078,PL BL MINT ANTSPTC RINSE,PRIVATE LABEL,ORAL HYGIENE PRODUCTS,MOUTHWASHES (ANTISEPTIC),500 ML


,WEEK_END_DATE,STORE_NUM,UPC,UNITS,VISITS,HHS,SPEND,PRICE,BASE_PRICE,FEATURE,DISPLAY,TPR_ONLY
0,2009-01-14 00:00:00,367,1111009477,13,13,13,18.07,1.39,1.57,0,0,1
1,2009-01-14 00:00:00,367,1111009497,20,18,18,27.80,1.39,1.39,0,0,0
2,2009-01-14 00:00:00,367,1111009507,14,14,14,19.32,1.38,1.38,0,0,0
3,2009-01-14 00:00:00,367,1111035398,4,3,3,14.00,3.50,4.49,0,0,1
4,2009-01-14 00:00:00,367,1111038078,3,3,3,7.50,2.50,2.50,0,0,0


,STORE_ID,STORE_NAME,ADDRESS_CITY_NAME,ADDRESS_STATE_PROV_CODE,MSA_CODE,SEG_VALUE_NAME,PARKING_SPACE_QTY,SALES_AREA_SIZE_NUM,AVG_WEEKLY_BASKETS
0,389,SILVERLAKE,ERLANGER,KY,17140,MAINSTREAM,408.0,46073,24766.807692
1,2277,ANDERSON TOWNE CTR,CINCINNATI,OH,17140,UPSCALE,NaN,81958,54052.519231
2,4259,WARSAW AVENUE,CINCINNATI,OH,17140,VALUE,NaN,48813,31177.333333
3,6379,KINGWOOD,KINGWOOD,TX,26420,MAINSTREAM,NaN,50237,20620.423077
4,6431,AT WARD ROAD,BAYTOWN,TX,26420,VALUE,350.0,43698,24321.942308


In [19]:
products.CATEGORY.unique()

array(['BAG SNACKS', 'ORAL HYGIENE PRODUCTS', 'COLD CEREAL',
       'FROZEN PIZZA'], dtype=object)

In [22]:
transactions.WEEK_END_DATE.unique().size

156

In [66]:
transactions = transactions.dropna(axis=0)
transactions['Discount'] = transactions.FEATURE | transactions.DISPLAY | transactions.TPR_ONLY
transactions['Discount_Percentage']= (transactions.BASE_PRICE - transactions.PRICE)/transactions.BASE_PRICE
transactions.head()

,WEEK_END_DATE,STORE_NUM,UPC,UNITS,VISITS,HHS,SPEND,PRICE,BASE_PRICE,FEATURE,DISPLAY,TPR_ONLY,Discount,Discount_Percentage
0,2009-01-14 00:00:00,367,1111009477,13,13,13,18.07,1.39,1.57,0,0,1,1,0.11465
1,2009-01-14 00:00:00,367,1111009497,20,18,18,27.80,1.39,1.39,0,0,0,0,0.00000
2,2009-01-14 00:00:00,367,1111009507,14,14,14,19.32,1.38,1.38,0,0,0,0,0.00000
3,2009-01-14 00:00:00,367,1111035398,4,3,3,14.00,3.50,4.49,0,0,1,1,0.22049
4,2009-01-14 00:00:00,367,1111038078,3,3,3,7.50,2.50,2.50,0,0,0,0,0.00000


####  Total sale, sale in dicount period, sale in normal period per category

In [168]:
Total_sale = transactions.merge(products[['UPC','CATEGORY']], on='UPC', how='left')\
             .groupby('CATEGORY').UNITS.sum().reset_index()

Total_sale_normal = transactions[transactions.Discount==0]\
                     .merge(products[['UPC','CATEGORY']], on='UPC', how='left')\
                     .groupby('CATEGORY').UNITS.sum().reset_index()\
                     .rename(columns={'UNITS':'UNITS_NORMAL'})

Total_sale_discount = transactions[transactions.Discount==1]\
                     .merge(products[['UPC','CATEGORY']], on='UPC', how='left')\
                     .groupby('CATEGORY').UNITS.sum().reset_index()\
                     .rename(columns={'UNITS':'UNITS_DISC'})

In [174]:
sales_combined = Total_sale\
                 .merge(Total_sale_normal, on='CATEGORY', how='left')\
                 .merge(Total_sale_discount, on='CATEGORY', how='left')
sales_combined

,CATEGORY,UNITS,UNITS_NORMAL,UNITS_DISC
0,BAG SNACKS,2601247,1490290,1110957
1,COLD CEREAL,5826783,3301659,2525124
2,FROZEN PIZZA,1341731,609736,731995
3,ORAL HYGIENE PRODUCTS,521140,282085,239055


#### Sales by Feature, Display, or TPR_Only

In [152]:
Total_sale_feature = transactions[transactions.FEATURE==1]\
                     .merge(products[['UPC','CATEGORY']], on='UPC', how='left')\
                     .groupby('CATEGORY').UNITS.sum()
Total_sale_display = transactions[transactions.DISPLAY==1]\
                     .merge(products[['UPC','CATEGORY']], on='UPC', how='left')\
                     .groupby('CATEGORY').UNITS.sum()
Total_sale_Only = transactions[transactions.TPR_ONLY==1]\
                    .merge(products[['UPC','CATEGORY']], on='UPC', how='left')\
                     .groupby('CATEGORY').UNITS.sum()

In [156]:
Total_sale_feature

CATEGORY
BAG SNACKS                111079
COLD CEREAL              1522949
FROZEN PIZZA              481440
ORAL HYGIENE PRODUCTS      63865
Name: UNITS, dtype: int64

### Normalised sales

In [158]:
Ndays_normal = transactions[transactions.Discount==0]\
                     .groupby(['WEEK_END_DATE','UPC'])\
                     .STORE_NUM.size().reset_index()\
                     .groupby('UPC').STORE_NUM.sum().reset_index()\
                     .merge(products[['UPC','CATEGORY']], on='UPC', how='left')\
                     .groupby('CATEGORY').STORE_NUM.sum()

Ndays_discount = transactions[transactions.Discount==0]\
                     .groupby(['WEEK_END_DATE','UPC'])\
                     .STORE_NUM.size().reset_index()\
                     .groupby('UPC').STORE_NUM.sum().reset_index()\
                     .merge(products[['UPC','CATEGORY']], on='UPC', how='left')\
                     .groupby('CATEGORY').STORE_NUM.sum()

In [159]:
Ndays_normal

CATEGORY
BAG SNACKS                88718
COLD CEREAL              129471
FROZEN PIZZA              77149
ORAL HYGIENE PRODUCTS     80203
Name: STORE_NUM, dtype: int64

#### Sales by Feature, Display, or TPR_Only (Normalised)

In [153]:
Ndays_feature = transactions[transactions.FEATURE==1]\
                     .groupby(['WEEK_END_DATE','UPC'])\
                     .STORE_NUM.size().reset_index()\
                     .groupby('UPC').STORE_NUM.sum().reset_index()\
                     .merge(products[['UPC','CATEGORY']], on='UPC', how='left')\
                     .groupby('CATEGORY').STORE_NUM.sum()
 
Ndays_display = transactions[transactions.DISPLAY==1]\
                     .groupby(['WEEK_END_DATE','UPC'])\
                     .STORE_NUM.size().reset_index()\
                     .groupby('UPC').STORE_NUM.sum().reset_index()\
                     .merge(products[['UPC','CATEGORY']], on='UPC', how='left')\
                     .groupby('CATEGORY').STORE_NUM.sum()

Ndays_Only = transactions[transactions.TPR_ONLY==1]\
                     .groupby(['WEEK_END_DATE','UPC'])\
                     .STORE_NUM.size().reset_index()\
                     .groupby('UPC').STORE_NUM.sum().reset_index()\
                     .merge(products[['UPC','CATEGORY']], on='UPC', how='left')\
                     .groupby('CATEGORY').STORE_NUM.sum()

#### How discount percentages impact sales

In [170]:
Ndays_feature = transactions[transactions.FEATURE==1]\
                     .groupby(['WEEK_END_DATE','UPC'])\
                     .STORE_NUM.size().reset_index()\
                     .groupby('UPC').STORE_NUM.sum().reset_index()\
                     .merge(products[['UPC','CATEGORY']], on='UPC', how='left')\
                     .groupby('CATEGORY').STORE_NUM.sum()
 
Ndays_display = transactions[transactions.DISPLAY==1]\
                     .groupby(['WEEK_END_DATE','UPC'])\
                     .STORE_NUM.size().reset_index()\
                     .groupby('UPC').STORE_NUM.sum().reset_index()\
                     .merge(products[['UPC','CATEGORY']], on='UPC', how='left')\
                     .groupby('CATEGORY').STORE_NUM.sum()

Ndays_Only = transactions[transactions.TPR_ONLY==1]\
                     .groupby(['WEEK_END_DATE','UPC'])\
                     .STORE_NUM.size().reset_index()\
                     .groupby('UPC').STORE_NUM.sum().reset_index()\
                     .merge(products[['UPC','CATEGORY']], on='UPC', how='left')\
                     .groupby('CATEGORY').STORE_NUM.sum()

In [ ]:
Ndays_display = transactions[transactions.DISPLAY==1]\
                     .merge(products[['UPC','CATEGORY']], on='UPC', how='left')\
                     .groupby('CATEGORY').UNITS.sum()
Ndays_Only = transactions[transactions.TPR_ONLY==1]\
                    .merge(products[['UPC','CATEGORY']], on='UPC', how='left')\
                     .groupby('CATEGORY').UNITS.sum()

#### Sales by Feature, Display, or TPR_Only (Normalised)

In [ ]:
# transactions=pd.read_excel("data/userdata.xlsx", sheet_name='dh Transaction Data')
# transactions1=transactions.copy(deep = True)
# transactions1=transactions1.dropna(axis='columns', how='all')
# transactions1.columns=transactions1.iloc[0].to_list()
# transactions1.drop(0, inplace=True)
# transactions1 =transactions1.to_csv ('data/products.csv', index = None, header=True)